In [1]:
import torch
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from glob import glob
from torch.utils.data import TensorDataset, DataLoader, Subset
from pytorch_lightning import loggers as pl_loggers
from models import FocalClassifier
from evaluation import evaluate
from torch import nn
from sklearn.model_selection import train_test_split

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [4]:
interictal_data = torch.load("data/interictal.pt")
preictal_data = torch.load("data/preictal.pt")

In [5]:
patient_ids = list(interictal_data.keys())
test_patient = patient_ids[0]

In [6]:
train_segments = torch.cat([patient_data for patient_id, patient_data in interictal_data.items() if patient_id!=test_patient])

In [7]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size=6, hidden_size=100, output_size=100):
        super().__init__()
        self.lstm_1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.lstm_2 = nn.LSTM(input_size=hidden_size, hidden_size=output_size, batch_first=True)

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

    def forward(self, x):
        N, L, H = x.shape
        x, (_, _) = self.lstm_1(x)
        _, (h_n, _) = self.lstm_2(x)
        return h_n.reshape((N, 1, self.output_size))

class DecoderLSTM(nn.Module):
    def __init__(self, seq_length=120, input_size=100, hidden_size=100, output_size=6):
        super().__init__()
        self.lstm_1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.lstm_2 = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, batch_first=True)
        self.output_layer = nn.Linear(in_features=hidden_size, out_features=output_size)
        

        self.seq_length = seq_length
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

    def forward(self, x):
        N, L, H = x.shape
        self.h_n, self.c_n = torch.zeros([1, N, self.hidden_size]).to(device), torch.zeros([1, N, self.hidden_size]).to(device)
        output = torch.zeros([N, self.seq_length, self.hidden_size]).to(device)
        for i in range(self.seq_length):
            x, (self.h_n, self.c_n) = self.lstm_1(x, (self.h_n, self.c_n))
            output[:, i, :] = x.squeeze()
        output = self.output_layer(output)
        return output

class AnomalyDetector(pl.LightningModule):
    def __init__(self, seq_length=120, input_size=6, hidden_size=100, encoder_output_size=100, lr=1e-3):
        super().__init__()
        self.seq_length = seq_length
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.encoder_output_size = encoder_output_size
    
        self.encoder = EncoderLSTM(input_size=input_size, hidden_size=hidden_size, output_size=encoder_output_size)
        self.decoder = DecoderLSTM(seq_length=seq_length, input_size=encoder_output_size, hidden_size=hidden_size, output_size=input_size)

        self.lr = 1e-3
        self.loss = nn.L1Loss()
        self.example_input_array = torch.rand((1, 120, 6))

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return (encoded, decoded)
    
    def training_step(self, batch, batch_idx):
        X = batch[0]
        encoded, decoded = self(X)
        loss = self.loss(decoded, X)
        self.log("Training Loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X = batch[0]
        encoded, decoded = self(X)
        loss = self.loss(decoded, X)
        self.log("Validation Loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.99)
        return {"optimizer":optimizer, "lr_scheduler":lr_scheduler} 

In [8]:
dataset = TensorDataset(train_segments)
train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=0.1)
train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)

model = AnomalyDetector()
tb_logger = pl_loggers.TensorBoardLogger("experiments/anomaly_detection")
trainer = pl.Trainer(gpus=1, max_epochs=200, logger=tb_logger)
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type        | Params | In sizes    | Out sizes  
--------------------------------------------------------------------
0 | encoder | EncoderLSTM | 124 K  | [1, 120, 6] | [1, 1, 100]
1 | decoder | DecoderLSTM | 162 K  | [1, 1, 100] | [1, 120, 6]
2 | loss    | L1Loss      | 0      | ?           | ?          
--------------------------------------------------------------------
286 K     Trainable params
0         Non-trainable params
286 K     Total params
1.145     Total estimated model params size (MB)


Epoch 1:   5%|▌         | 41/815 [00:04<01:22,  9.37it/s, loss=0.0742, v_num=7]

In [ ]:
np.random.choice(range(len(train_segments)), )

34975